# Optimisation of planing
### Table of Contents

* [1. Introduction](#1.introduction)
* [2. First basic problem](#fbp)
    * [2.1. Math and first step with pyomo for solving the problem](#math)
    * [2.2. Variables ](#variables)
    * [2.3. Constraints](#constraints)
* [3. Extensions of this operation problem ](#temporal)
    * [3.1. Linear temporal coupling with ramp constraints ](#ramp)
    * [3.2. Linear spatial coupling with spatial constraints - Problem Op3 Multi-Area -](#spatial)
* [4. Storage operation ](#storage)
    * [4.1. Optimisation of a storage market participation](#storageMarket)
    * [4.2. Simultaneous optimisation of storage and electric system](#storagecoupling)

## 1. Introduction <a class="anchor" id="1.introduction"></a>
This document contains a description of optimisation tools for electric system planing simulation.
If you are only interested in operation you might prefer to go back to file optim-Operation.ipynb.
In files BasicFunctionalities/input-XXX.ipynb you can learn to understand input data (consumption, availability).

(Text below is almost the same as for operation)

This document will gives a chance to understand
 - how to do the optimisation of planing of an electric system under the hourly operation constraint
 - understand the mathematical formulation of the optimisation problem
 - learn to analyse de results of the optimisation and in particular the Lagrange multipliers
 - get in touch with pyomo (a python package to write optimisation problems)

It proposes to enter the subject by increasing progressively
the number of variables and constraints in the optimisation problem, hence moving toward more realism through the document, introducing:
 - ramp constraints that implies a simple temporal coupling
 - spatially indexed variables and congestion constraints that implies a simple spatial coupling.
 - storage constraints that implies a temporal coupling

It relies on different test cases that allow to
 - consider different production means (nuclear, CCG, solar, onshore wind power, offshore wind power, hydro, curtailement of consumption, storage)
 - consider different meteorological years for France
 - consider different countries in the multi-zone case (France, Germany, GB, Spain)

If, after reading this file, you want to build your own pyomo model you can go to optim-Planing-Advanced.ipynb.

In [1]:
#region importation of modules
import os
if os.path.basename(os.getcwd())=="BasicFunctionalities":
    os.chdir('..') ## to work at project root  like in any IDE
InputFolder='Data/input/'

import qgrid # great package https://github.com/quantopian/qgrid
import numpy as np
import pandas as pd
import csv
import docplex
import datetime
import copy
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn import linear_model
import sys

from functions.f_planingModels import *
from functions.f_optimization import *
from functions.f_graphicalTools import *
# Change this if you have other solvers obtained here
## https://ampl.com/products/solvers/open-source/
## for eduction this site provides also several professional solvers, that are more efficient than e.g. cbc
#endregion

#region Solver location definition
myhost = os.uname()[1]
if (myhost=="jupyter-sop"):
    ## for https://jupyter-sop.mines-paristech.fr/ users, you need to
    #  (1) run the following to loanch the license server
    os.system("/opt/mosek/9.2/tools/platform/linux64x86/bin/lmgrd -c /opt/mosek/9.2/tools/platform/linux64x86/bin/mosek.lic -l lmgrd.log")
    #  (2) definition of license
    os.environ["MOSEKLM_LICENSE_FILE"] = '@jupyter-sop'

solver= 'mosek' ## no need for solverpath with mosek.
BaseSolverPath='/Users/robin.girard/Documents/Code/Packages/solvers/ampl_macosx64'
sys.path.append(BaseSolverPath)

solvers= ['gurobi','knitro','cbc'] # 'glpk' is too slow 'cplex' and 'xpress' do not work
solverpath= {}
for solver in solvers : solverpath[solver]=BaseSolverPath+'/'+solver
cplexPATH='/Applications/CPLEX_Studio1210/cplex/bin/x86-64_osx'
sys.path.append(cplexPATH)
solverpath['cplex']=cplexPATH+"/"+"cplex"
solver = 'mosek'
#endregion


## 2. First basic problem <a class="anchor" id="fbp"></a>
### 2.1. Math and first step with pyomo for solving problem<a class="anchor" id="math"></a>
Before you start with the math, you should
 - be expert in optimisation of operation, your can learn things in optim-Operation.ipynb
 - be familiar with annualized cost of electricity (variable and fixed cost) and levelised cost of energy.
You can read [my post here on lcoe](https://www.energy-alternatives.eu/2020/08/20/decomposition-lcoe.html) (in french, if there are people that would be willing to contribute to a translation that would be great !!)
and also [this one on the role of power and enery in system cost](https://www.energy-alternatives.eu/2020/05/07/mix-de-production-delectricite-energie-et-puissance.html).

\begin{align}
&\text{Cost function }& &\min_{x,\bar{x}}   \sum_i \left ( \beta_i\bar{x_i}+ \sum_t\pi_i x_{it}\right ) \;\;\; & & \pi_i \text{ marginal cost, }\beta_i \text{fixed annualized cost}\\
&\text{Power limit }   & &\text{ s.t.} \;\; 0 \leq x_{it}\leq a_{it} \bar{x_i} & &\bar{x_i} \text{ installed power, }  a_{it} \text{ availability}\\
&\text{Meet demand }   & & \sum_i x_{it} \geq  C_t  && C_t \text{ Consumption}\\
\end{align}

In [2]:
#region I - Simple single area : loading parameters
Zones="FR" ; year=2013
#### reading areaConsumption availabilityFactor and TechParameters CSV files
areaConsumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(Zones)+'.csv',sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',sep=',',decimal='.',skiprows=0)
TechParameters = pd.read_csv(InputFolder+'Planing-Simple_TECHNOLOGIES.csv',sep=',',decimal='.',skiprows=0,comment="#")

#### Selection of subset
Selected_TECHNOLOGIES=['OldNuke','CCG'] #you can add technologies here
availabilityFactor=availabilityFactor[ availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
TechParameters=TechParameters[TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
#endregion

#region I - Simple single area  : Solving and loading results
model = GetElectricSystemModel_PlaningSingleNode(areaConsumption,availabilityFactor,TechParameters)

if solver in solverpath :  opt = SolverFactory(solver,executable=solverpath[solver])
else : opt = SolverFactory(solver)
results=opt.solve(model)
## result analysis
Variables=getVariables_panda(model)
Variables['capacity']
#pour avoir la production en KWh de chaque moyen de prod chaque heure
production_df=Variables['energy'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='energy')
production_df.sum(axis=0)/10**6 ### energies produites TWh

energyCosts_df=Variables['energyCosts'].set_index("TECHNOLOGIES")
capacityCosts_df=Variables['capacityCosts'].set_index("TECHNOLOGIES")
costs_df=pd.concat([energyCosts_df,capacityCosts_df],axis=1)
costs_df["totalCosts"]=costs_df["energyCosts"]+costs_df["capacityCosts"]

#endregion

## 2.2 Analysing results : lagrange multipliers <a class="anchor" id="optiofope"></a>

 Verify that the sum of market prices allows all actors to cover fixed and marginal cost.
do they earn more ? why ?

In [ ]:
#region I - Simple single area  : visualisation and lagrange multipliers
### representation des résultats
fig=MyStackedPlotly(x_df=production_df.index,
                    y_df=production_df[list(Selected_TECHNOLOGIES)],
                    Names=list(Selected_TECHNOLOGIES))
fig.update_layout(title_text="Production électrique (en KWh)", xaxis_title="heures de l'année")
fig.show()
#plotly.offline.plot(fig, filename='file.html') ## offline
#fig2.show()

#### lagrange multipliers
Constraints= getConstraintsDual_panda(model)

# Analyse energyCtr
energyCtrDual=Constraints['energyCtr']; energyCtrDual['energyCtr']=energyCtrDual['energyCtr']*1000000
energyCtrDual
round(energyCtrDual.energyCtr,2).unique()

# Analyse CapacityCtr
CapacityCtrDual=Constraints['CapacityCtr'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='CapacityCtr')*1000000;
round(CapacityCtrDual,2)
round(CapacityCtrDual.OldNuke,2).unique() ## if you increase by Delta the installed capacity of nuke you decrease by xxx the cost when nuke is not sufficient
round(CapacityCtrDual.CCG,2).unique() ## increasing the capacity of CCG as no effect on prices
#endregion

## 3. Extensions of this operation problem <a class="anchor" id="temporal"></a>
### 3.1.  Linear temporal coupling with ramp constraints <a class="anchor" id="ramp"></a>
In the this section, we will increase the complexity of the problem
given in Section 2 and add : dependency on area z (country),
 a congestion constraint, ramp constraints.

\begin{align}
&\text{Cost function }& &\min_{x}  \sum_z \sum_t \sum_i \pi_{iz} x_{itz}\;\;\; & & \pi_{iz} \text{ marginal cost}\\
&\text{Power limit }   & &\text{ s.t.} \;\; 0 \leq x_{itz}\leq a_{itz} \bar{x_{iz}} & &\bar{x_{iz}} \text{ installed power, }  a_{itz} \text{ availability}\\
&\text{Meet demand }   & & \sum_i x_{itz} \geq  C_{tz}  && C_{tz} \text{ Consumption}\\
&\text{Stock limit }   & &\sum_t x_{it}\leq E_i && E_i=\bar{x_i}*N_i \text{ Energy capacity limit}\\
&\text{ramp limit }   & &rc^-_i *x_{it}\leq x_{it}-x_{i(t+1)}\leq rc^+_i *x_{it} && rc^+_i rc^-_i\text{ ramp limit}\\
\end{align}


In [ ]:
#region II - Ramp Single area : loading parameters loading parameterscase with ramp constraints
Zones="FR"
year=2013
Selected_TECHNOLOGIES={'OldNuke','CCG'} #you'll add 'Solar' ,'WindOnShore' after
#### reading CSV files
areaConsumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
TechParameters = pd.read_csv(InputFolder+'Planing-RAMP1_TECHNOLOGIES.csv',sep=',',decimal='.',skiprows=0,comment="#")

#### Selection of subset
availabilityFactor=availabilityFactor[availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
TechParameters=TechParameters[TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
#endregion

#region II - Ramp Single area : solving and loading results
model = GetElectricSystemModel_PlaningSingleNode(areaConsumption,availabilityFactor,TechParameters)
opt = SolverFactory(solver)
results=opt.solve(model)
Variables=getVariables_panda(model)

#pour avoir la production en KWh de chaque moyen de prod chaque heure
production_df=Variables['energy'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='energy')
production_df.sum(axis=0)/10**6 ### energies produites TWh
print(Variables['energyCosts']) #pour avoir le coût de chaque moyen de prod à l'année
#endregion

#region II - Ramp Single area : visualisation and lagrange multipliers
fig=MyStackedPlotly(x_df=production_df.index,
                    y_df=production_df[list(Selected_TECHNOLOGIES)],
                    Names=list(Selected_TECHNOLOGIES))
fig.update_layout(title_text="Production électrique (en KWh)", xaxis_title="heures de l'année")
#plotly.offline.plot(fig, filename='file.html') ## offline
fig.show()
#fig2.show()

#### lagrange multipliers
Constraints= getConstraintsDual_panda(model)

# Analyse energyCtr
energyCtrDual=Constraints['energyCtr']; energyCtrDual['energyCtr']=energyCtrDual['energyCtr']*1000000
energyCtrDual
round(energyCtrDual.energyCtr,2).unique()

# Analyse CapacityCtr
CapacityCtrDual=Constraints['CapacityCtr'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='CapacityCtr')*1000000;
round(CapacityCtrDual,2)
round(CapacityCtrDual.OldNuke,2).unique() ## if you increase by Delta the installed capacity of nuke you decrease by xxx the cost when nuke is not sufficient
round(CapacityCtrDual.CCG,2).unique() ## increasing the capacity of CCG as no effect on prices
#endregion

### 3.2.  Linear spatio-temporal coupling with ramp+spatial constraints <a class="anchor" id="spatial"></a>

Math here are in 3.1

In [ ]:
#region III - Ramp multiple area : loading parameters
Zones="FR_DE_GB_ES"
year=2016
Selected_AREAS={"FR","DE"}
Selected_TECHNOLOGIES={'CCG', 'OldNuke' } #'NewNuke', 'HydroRiver', 'HydroReservoir','WindOnShore', 'WindOffShore', 'Solar', 'Curtailement'}
Selected_TECHNOLOGIES={'CCG', 'OldNuke', 'NewNuke', 'HydroRiver', 'HydroReservoir',
       'WindOnShore', 'WindOffShore', 'Solar', 'Curtailement'}
#### reading CSV files
areaConsumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
TechParameters = pd.read_csv(InputFolder+'Planing_MultiNode_DE-FR_TECHNOLOGIES_AREAS.csv',sep=',',decimal='.',skiprows=0,comment="#")

ExchangeParameters = pd.read_csv(InputFolder+'Hypothese_DE-FR_AREAS_AREAS.csv',sep=',',decimal='.',skiprows=0,comment="#")
#### Selection of subset
TechParameters=TechParameters[TechParameters.AREAS.isin(Selected_AREAS)&TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
areaConsumption=areaConsumption[areaConsumption.AREAS.isin(Selected_AREAS)]
availabilityFactor=availabilityFactor[availabilityFactor.AREAS.isin(Selected_AREAS)& availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
#endregion

In [ ]:
#region III - Ramp multiple area : solving and loading results
### small data cleaning
availabilityFactor.availabilityFactor[availabilityFactor.availabilityFactor>1]=1
model = GetElectricSystemModel_PlaningMultiNode(areaConsumption,availabilityFactor,TechParameters,ExchangeParameters)
opt = SolverFactory(solver)
results=opt.solve(model)
Variables=getVariables_panda(model)
Variables['capacity']
Variables['capacity']
production_df=EnergyAndExchange2Prod(Variables)
fig=MyAreaStackedPlot(production_df,Selected_TECHNOLOGIES=Selected_TECHNOLOGIES+Selected_AREAS)
fig.show()
#plotly.offline.plot(fig, filename='file.html') ## offline

production_df=Variables['energy'].pivot(index=["TIMESTAMP","AREAS"], columns='TECHNOLOGIES', values='energy')
production_df.sum(axis=0)/10**6 ### energies produites TWh
production_df.groupby(by="AREAS").sum()/10**6 ### energies produites TWh

energyCosts_df=Variables['energyCosts'].set_index(["AREAS","TECHNOLOGIES"])
capacityCosts_df=Variables['capacityCosts'].set_index(["AREAS","TECHNOLOGIES"])
costs_df=pd.concat([energyCosts_df,capacityCosts_df],axis=1)
costs_df["totalCosts"]=costs_df["energyCosts"]+costs_df["capacityCosts"]
costs_df.groupby("AREAS")["totalCosts"].sum()

Variables["exchange"].head()
Variables["exchange"].exchange.sum()
Constraints= getConstraintsDual_panda(model)
Constraints.keys()
#endregion

## 4. Storage operation <a class="anchor" id="storage"></a>
### 4.1. Optimisation of a storage market participation <a class="anchor" id="storageMarket"></a>
Just have a look at optim-Storage.ipynb

### 4.2. Simultaneous optimisation of storage and electric system <a class="anchor" id="storagecoupling"></a>

This section is a bit more difficult that the other one. You can skip the math if you're only interested in using the storage optimisation tool.

In [ ]:
#region IV Ramp+Storage single area : loading parameters
Zones="FR"
year=2013

Selected_TECHNOLOGIES={'CCG', 'OldNuke', 'WindOnShore',"Curtailement"}

#### reading CSV files
areaConsumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
TechParameters = pd.read_csv(InputFolder+'Planing-RAMP1_TECHNOLOGIES.csv',sep=',',decimal='.',skiprows=0,comment="#")


#### Selection of subset
availabilityFactor=availabilityFactor[ availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
TechParameters=TechParameters[TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]

p_max=10000
StorageParameters={"p_max" : p_max , "c_max": p_max*10,"efficiency_in": 0.9,"efficiency_out" : 0.9}
#endregion

In [ ]:
#region IV Ramp+Storage single area : solving and loading results
res= GetElectricSystemModel_PlaningSingleNode_with1Storage(areaConsumption,availabilityFactor,
                                                      TechParameters,StorageParameters)

Variables = getVariables_panda(res['model'])
Constraints = getConstraintsDual_panda(res['model'])
areaConsumption = res["areaConsumption"]

production_df=Variables['energy'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='energy')
production_df.loc[:,'Storage'] = areaConsumption["Storage"]### put storage in the production time series
production_df.sum(axis=0)/10**6 ### energies produites TWh
production_df[production_df>0].sum(axis=0)/10**6 ### energies produites TWh
production_df.max(axis=0)/1000 ### Pmax en GW

Selected_TECHNOLOGIES_Sto=list(Selected_TECHNOLOGIES)
Selected_TECHNOLOGIES_Sto.append("Storage")
fig=MyStackedPlotly(x_df=production_df.index,
                    y_df=production_df[Selected_TECHNOLOGIES_Sto],
                    Names=Selected_TECHNOLOGIES_Sto)
fig.update_layout(title_text="Production électrique (en KWh)", xaxis_title="heures de l'année")
fig.show()
#plotly.offline.plot(fig, filename='file.html') ## offline
stats=res["stats"]
#endregion

### 4.3. Simultaneous optimisation of storage+spatial and electric system <a class="anchor" id="storagecouplingSpatial"></a>

In [ ]:
#region VI Ramp+Storage Multi area : loading parameters
Zones="FR_DE_GB_ES"
year=2016
Selected_AREAS={"FR","DE"}
Selected_TECHNOLOGIES={'CCG', 'OldNuke' } #'NewNuke', 'HydroRiver', 'HydroReservoir','WindOnShore', 'WindOffShore', 'Solar', 'Curtailement'}

#### reading CSV files
TechParameters = pd.read_csv(InputFolder+'Planing_MultiNode_DE-FR_TECHNOLOGIES_AREAS.csv',sep=',',decimal='.',comment="#")
areaConsumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)

ExchangeParameters = pd.read_csv(InputFolder+'Hypothese_DE-FR_AREAS_AREAS.csv',sep=',',decimal='.',skiprows=0,comment="#")
#### Selection of subset
TechParameters=TechParameters[TechParameters.AREAS.isin(Selected_AREAS)&TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
areaConsumption=areaConsumption[areaConsumption.AREAS.isin(Selected_AREAS)]
availabilityFactor=availabilityFactor[availabilityFactor.AREAS.isin(Selected_AREAS)& availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]

p_max=10000

StorageParameters=pd.DataFrame([])
for AREA in Selected_AREAS :
    StorageParameters_ = {"AREA": AREA, "p_max": p_max, "c_max": p_max * 10, "efficiency_in": 0.9,
                          "efficiency_out": 0.9}
    StorageParameters=StorageParameters.append(pd.DataFrame([StorageParameters_]))

#endregion

In [ ]:
#region VI Ramp+Storage multi area : solving and loading results
res= GetElectricSystemModel_PlaningMultiNode_with1Storage(areaConsumption,availabilityFactor,
                                                      TechParameters,ExchangeParameters,StorageParameters)

Variables = getVariables_panda(res['model'])
Constraints = getConstraintsDual_panda(res['model'])
areaConsumption = res["areaConsumption"]
#Variables['energy'].loc['Storage'] = areaConsumption["Storage"]

#areaConsumption['Storage'].range()
model= res["model"]
stats=res["stats"]
#endregion


## 5. System with a lot of renewable <a class="anchor" id="storage"></a>

In [ ]:
#region V Case Storage + CCG + PV + Wind (Ramp+Storage single area) : loading parameters
Zones="FR"
year=2013

Selected_TECHNOLOGIES={'CCG', 'WindOnShore','Solar',"Curtailement"}

#### reading CSV files
areaConsumption = pd.read_csv(InputFolder+'areaConsumption'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
availabilityFactor = pd.read_csv(InputFolder+'availabilityFactor'+str(year)+'_'+str(Zones)+'.csv',
                                sep=',',decimal='.',skiprows=0)
TechParameters = pd.read_csv(InputFolder+'Planing-RAMP1_TECHNOLOGIES.csv',sep=',',decimal='.',skiprows=0,comment="#")


#### Selection of subset
availabilityFactor=availabilityFactor[ availabilityFactor.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
TechParameters=TechParameters[TechParameters.TECHNOLOGIES.isin(Selected_TECHNOLOGIES)]
TechParameters.keys()
TechParameters.loc[TechParameters.TECHNOLOGIES=='Solar','minCapacity']=90000
TechParameters.loc[TechParameters.TECHNOLOGIES=='WindOnShore','minCapacity']=12000
TechParameters[['TECHNOLOGIES','minCapacity','maxCapacity']]
p_max=10000 ## storage capacity is not optimized
StorageParameters={"p_max" : p_max , "c_max": p_max*10,"efficiency_in": 0.9,"efficiency_out" : 0.9}
#endregion

#region V Case Storage + CCG + PV + Wind (Ramp+Storage single area) : solving and loading results
res= GetElectricSystemModel_PlaningSingleNode_with1Storage(areaConsumption,availabilityFactor,
                                                      TechParameters,StorageParameters)
Variables = getVariables_panda(res['model'])
Constraints = getConstraintsDual_panda(res['model'])
areaConsumption = res["areaConsumption"]

production_df=Variables['energy'].pivot(index="TIMESTAMP",columns='TECHNOLOGIES', values='energy')
production_df.loc[:,'Storage'] = areaConsumption["Storage"]### put storage in the production time series
production_df[production_df>0].sum(axis=0)/10**6 ### energies produites TWh
production_df.max(axis=0)/1000 ### Pmax en GW
Variables['capacity'] ### capacity en MW


Selected_TECHNOLOGIES_Sto=list(Selected_TECHNOLOGIES)
Selected_TECHNOLOGIES_Sto.append("Storage")
fig=MyStackedPlotly(x_df=production_df.index,
                    y_df=production_df[Selected_TECHNOLOGIES_Sto],
                    Names=Selected_TECHNOLOGIES_Sto)
fig.update_layout(title_text="Production électrique (en KWh)", xaxis_title="heures de l'année")
plotly.offline.plot(fig, filename='file.html') ## offline
fig.show()
stats=res["stats"]
#endregion


TODO here  : add electrolysers